In [98]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [71]:
from jellyfish.core import Client
from jellyfish.history_loader import load_candles_history
from jellyfish import utils
from datetime import datetime, timedelta

utils.disable_warnings()

pair = 'btcusdt'
mid_dt = datetime(year=2021, month=4, day=25)
size = 5000 * 2

frame = load_candles_history(Client(), pair, end_dt=mid_dt, interval='1m', candles_num=size)

  0%|          | 0/10 [00:00<?, ?it/s]

In [78]:
from jellyfish import indicator
import numpy as np
import pandas as pd

def preprocess(frame: pd.DataFrame):
    returns = frame.Close / frame.Open
    hurst = indicator.hurst(returns, kind=indicator.HURST_CHANGE, simplified=False)

    th = hurst[100:].mean()
    th_eps = hurst[100:].std() * 1e-3
    th_eps = hurst[100:].std() * 3.5e-2

    frame['Persistent'] = (hurst > th + th_eps).astype(np.int32)
    frame['Reversal'] = (hurst < th - th_eps).astype(np.int32)
    frame['RandomWalk'] = (np.abs(hurst - th) < th_eps).astype(np.int32)
    frame['i_Hurst'] = hurst
    return frame

frame = preprocess(frame)

  0%|          | 0/9760 [00:00<?, ?it/s]

In [79]:
from jellyfish.transform.sampling import _generic_sampling, DEFAULT_SAMPLING_AGG
from jellyfish.core import SmaCross, Backtest

condition = lambda ohlc: ohlc.Persistent.max() + ohlc.Reversal.max() > 1
frame1 = _generic_sampling(frame.reset_index(), condition, DEFAULT_SAMPLING_AGG).set_index('Date')

  0%|          | 0/9859 [00:00<?, ?it/s]

In [104]:
class MyStrategy(SmaCross):
    rsi_period = 5

    def init(self):
        self.rsi = self.I(indicator.rsi, self.data.Close, self.rsi_period)
        super(MyStrategy, self).init()

    def next(self):
        if self.rsi < 50:
            self.position.close()
        else:
            super(MyStrategy, self).next()

bt = Backtest(frame1, MyStrategy, trade_on_close=True)
stats = bt.optimize(
    n1=10,
    n2=20,
    rsi_period=10
)
bt.plot(open_browser=True)
stats

Backtest.optimize:   0%|          | 0/1 [00:00<?, ?it/s]

Start                     2021-04-18 06:05:00
End                       2021-04-25 03:00:00
Duration                      6 days 20:55:00
Exposure Time [%]                   24.076809
Equity Final [$]                950711.664104
Equity Peak [$]                 1008792.67006
Return [%]                          -4.928834
Buy & Hold Return [%]              -14.528594
Return (Ann.) [%]                  -86.665048
Volatility (Ann.) [%]                4.149941
Sharpe Ratio                        -7.085944
Calmar Ratio                       -37.054617
Max. Drawdown [%]                   -7.070477
Avg. Drawdown [%]                   -3.035111
Max. Drawdown Duration        4 days 01:50:00
Avg. Drawdown Duration        2 days 02:33:00
# Trades                                   15
Win Rate [%]                             40.0
Best Trade [%]                       0.923247
Worst Trade [%]                     -3.619569
Avg. Trade [%]                      -0.349134
Max. Trade Duration           0 da

In [113]:
from jellyfish.core import Strategy


class RsiStrategy(Strategy):
    rsi_period = 5
    th = 50
    th_eps = 5

    def init(self):
        self.rsi = self.I(indicator.rsi, self.data.Close, self.rsi_period)

    def next(self):
        if self.rsi < self.th - self.th_eps:
            if not self.position.is_short:
                self.position.close()
                self.sell()

        elif self.rsi > self.th + self.th_eps:
            if not self.position.is_long:
                self.position.close()
                self.buy()

bt = Backtest(frame1, RsiStrategy, trade_on_close=True)
stats = bt.optimize(
    rsi_period=10
)
bt.plot(open_browser=True)
stats

Backtest.optimize:   0%|          | 0/1 [00:00<?, ?it/s]

Start                     2021-04-18 06:05:00
End                       2021-04-25 03:00:00
Duration                      6 days 20:55:00
Exposure Time [%]                   99.704579
Equity Final [$]               1112631.549824
Equity Peak [$]                1124514.618908
Return [%]                          11.263155
Buy & Hold Return [%]              -14.528594
Return (Ann.) [%]                  480.496708
Volatility (Ann.) [%]              532.920796
Sharpe Ratio                         6.580531
Calmar Ratio                          59.5746
Max. Drawdown [%]                    -10.2504
Avg. Drawdown [%]                   -1.670771
Max. Drawdown Duration        2 days 23:57:00
Avg. Drawdown Duration        0 days 08:28:00
# Trades                                   32
Win Rate [%]                           46.875
Best Trade [%]                       7.376742
Worst Trade [%]                     -2.806667
Avg. Trade [%]                        0.28691
Max. Trade Duration           0 da